**Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
import urllib
import pickle
from matplotlib import pyplot as plt
%matplotlib inline
import cv2
import warnings
warnings.filterwarnings('ignore')
import config as config

## Functions

**Telegram and create data**

In [ ]:
def image_url_to_numpy_array_urllib(url, format=None):
    """
    :param url: url of a photo
    :param format: the format of the photo
    :return: return array of the resized image, 
    if the image ratio is too rectangular it return a message.
    """

    ## read as HTTPResponse 
    resp = urllib.request.urlopen(url)
    ## read as 1D bytearray
    resp_byte_array = resp.read()
    ## returns a bytearray object which is a mutable sequence of integers in
    # the range 0 <=x< 256
    mutable_byte_array = bytearray(resp_byte_array)
    ## read as unsigned integer 1D numpy array
    image = np.asarray(mutable_byte_array, dtype = "uint8")
    ## To decode the 1D image array into a 2D format with RGB color components
    # we make a call to cv2.imdecode
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # filter images with too rectangular ratio
    if (image.shape[0] > (2 * image.shape[1])) or (image.shape[1] > (2 * image.shape[0])):
        return 'Image ratio is too rectangular'
    image = cv2.resize(image, (224,224))
    if format == 'BGR':
        ## return BGR format array
        return image
    ## cv2.imdecode converted array into BGR format, convert it to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # return the image
    return image

**Train models**

In [ ]:
def load_data():
    """
    :return: X - an array that includes all the images.
             y - an array that includes a binary vector of the 10 most common keywords
                 for each image. 
    """
    # Upload the files
    X = pd.read_pickle(f'X1.pkl')
    y = pd.read_pickle(f'y1.pkl')

    for i in range(1,13):
        tmp_X = pd.read_pickle(f'X{i + 1}.pkl')
        tmp_y = pd.read_pickle(f'y{i + 1}.pkl')
        X = np.concatenate((X, tmp_X), axis = 0)
        y = y.append(tmp_y, ignore_index = True)

    X = np.stack(X, axis = 0)

    print('X shape', X.shape, '\ny shape', y.shape)
    return X, y

In [ ]:
def plot_over_epoches(history):
    """
    :param history: the model history after training.
    The function plot accuracy over epoches and loss over epoches, 
    both for training and validation sets.
    """

    fig, ax = plt.subplots(1, 1, figsize=(7, 4))
    ax.plot(history.history['accuracy'], color='salmon', label="Training accuracy")
    ax.plot(history.history['val_accuracy'], color='purple', label="validation accuracy")
    ax.set_title("Accuracy over epoches")
    ax.grid(alpha=0.2)
    legend = ax.legend(loc='best', shadow=True)
    plt.tight_layout()
    plt.show()
    
    fig, ax = plt.subplots(1, 1, figsize=(7, 4))
    ax.plot(history.history['loss'], color='salmon', label="Training loss")
    ax.plot(history.history['val_loss'], color='purple', label="validation loss")
    ax.set_title("Loss over epoches")
    ax.grid(alpha=0.2)
    legend = ax.legend(loc='best', shadow=True)
    plt.tight_layout()
    plt.show()

In [ ]:
def pic(X, y_pred, col_dict): 
    """
    :param X: an array of the image.
    :param y_pred: the predicted keywords for the image
    :param col_dict: dictionary of the keywords
    The function plot the image with the predicted keywords and the 
    confidence of the keyword.
    """
    
    print('-------------------------------------------')
    for i in range(10):
        confidence = (round(y_pred[i], 4) * 100)
        if confidence > 20:
            print(f'{col_dict[i]} - {confidence}%')
    plt.figure(figsize = (6,6))
    plt.imshow(X)
    plt.axis('off')
    plt.show()
    print('-------------------------------------------', end='\n\n\n')